In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from lumicks import pylake

%matplotlib inline

# Loading files

First we need to import the data files exported from Bluelake, in this case we are going to compare 2 fd curves previously recorded.

In [ ]:
file1 = pylake.File("data/20180611-142601 FD Curve 1.h5")
file2 = pylake.File("data/20180611-142629 FD Curve 2.h5")

# Inspect file content
To check the content of the file we can use the function 
```
print("file name")
``` 
to list all the items that are stored inside. In this case the file contains the FD curve item with the corresponding force calibration values.

In [ ]:
print(file1)

# Visualize data

Once we know the name of the items we want to visualize, we can load them from the file and display them.

By default, the FD curve will be displayed with the Distance 1 on the X axis and Force 2 on the  Y axis:
```
fd1.plot_scatter(label="Fd 1")
```

but any channel can be chosen: 
```
fd2.with_channels(force='1x', distance='1').plot_scatter(label="Fd 2")
```

In [ ]:
# load the Fd items from file
fd1 = file1.fdcurves["1"]
fd2 = file2.fdcurves["2"]
plt.figure(figsize=(16,5))
# plot with default channels
fd1.plot_scatter(label="Fd 1")
# plot with custom channel selection
fd2.with_channels(force='2x', distance='1').plot_scatter(label="Fd 2")
plt.legend();

# Data analysis

We can compare the recorded data with the teoretical models

### Odijk Worm-like chain model

$$ d = Lc \bigg (1 - \frac{1}{2} \sqrt{ \frac{kT}{F Lp}} + \frac{F}{S} \bigg)$$

### Freely-Jointed Chain

$$ d = Lc \bigg [ \coth \Big ({\frac{2FLp}{kT} \Big)  - \frac{kT}{2FLp}} \bigg] \bigg( 1 + \frac{F}{S}\bigg) $$ 





| Parameter| Description | 
|---|------------------------|
|d  | extension (um)         |
|F  | Force (pn)             |
|Lp | Persistence lenght (nm)|
|Lc | Contour lenght (um)    |
|S  | Stretching modulus (p) |
|kT | Boltzmann's costant times the temperature (optional, by default 4.11 pN nm)|









In [ ]:
def fOdijk(F,Lp,Lc,S,kT=4.11): 
    return Lc * (1 - 1./2*np.sqrt(kT/(F*Lp)) + F/S)

def fFJC(F,Lp,Lc,S,kT=4.11): 
    return Lc * (1/np.tanh((F*Lp)/kT) - kT/(F*Lp)) * ( 1 + F/S)

# generate our force range on interest 
f = np.linspace(fd1.f.data.min(),fd1.f.data.max(),fd1.f.data.size)

# generate the reference curves
ref1 = fOdijk(f,Lp=50,Lc=16,S=1000)
ref2 = fFJC(f,Lp=1.5,Lc=16,S=1000)

# plot our data alongside the reference curves
plt.figure(figsize=(16,5))
fd1.plot_scatter(label="Fd 1")
plt.plot(ref1,f,'r',label="Odijk Worm-like Chain");
#plt.plot(ref2,f,'g',label="Freely-Jointed Chain");
plt.legend();

# Force calibration
We can also display the calibration values which are always stored alonside the items: 

In [ ]:
# Print the list of calibration paramenters
list(file1["Calibration"]["19"]["Force 2x"].h5.attrs.items())